This notebook contains functions implementing the basic operations in Hyperdimensional Computing


#### Libraries

In [5]:
import numpy as np

### Global variables

In [6]:
Dict_HD = {}

### Initializing memory

In [7]:
def init(Dim = 10000):
    "Initialize vector 'null' and symbol matrix"
    global Dict, Memory, N, Dict_defs, feature_vectors, thr
    N = Dim     # Vectors' lenght
    Dict = {}; Memory = [[],[],[]]
    Dict_defs = {}; feature_vectors = []
    thr = 0.45 * N # Similarity threshold

init()

#### Random vectors and addition

In [8]:
def SparseBitString(n):
    """This function generates a random binary vector
    n: length of vector
    BitString: Binary vector"""
    # Generates random address indexes for setting them to 1
    Address = np.random.choice( np.arange(n), int(n/2), replace = False) 
    # Initialize binary vector
    BitString = np.zeros(n, dtype = np.int8)
    # Set to 1 the address generated
    BitString[Address] = 1
    return BitString

def ADD(*arg):
    " General function for vectors addition, it assumes that the arguments are 'HDvector' objects"
    if len(arg) == 1 and type(arg[0]) is list:
        arg = arg[0]
    len_0 = arg[0].lenght
    Sum = np.zeros(len_0) # Initialize sum vector
    n = len(arg) # Number of vectors to add
    for vec in arg:
        assert vec.lenght == len_0  # Are all dimensions equal??
        Sum = Sum + vec.vec         # 'normal' sum
    Sum = Sum / n                   # Average
    Sum[Sum > 0.5] = 1              #Thresholding
    Sum[Sum == 0.5] = SparseBitString(len_0)[Sum == 0.5]  # If average equal to 0.5 -> select a random value (0 or 1)
    return HDvector(Sum.astype(np.int8))

def contamina_vec(array, dif_bits):
    "Given a binary array and a integer (dif_bits), it retusn another array at a hamming distance of dif_bits"
    new_array = array.copy()
    index_randm = np.random.randint(0, len(array)-1, dif_bits)
    for i in np.unique(index_randm):
        if new_array[i] == 0:
            new_array[i] = 1
        else:
            new_array[i] = 0
    return new_array

### HD vector class

In [9]:
# Hyperdimensional vectors class
class HDvector (object):
    # We can initialize our vector with its content (array) or its length (content random)
    def __init__ (self, vec_or_len, label = None, pointer = None):
        global Memory   # We use this global variable
    
        if type(vec_or_len) is int: # Initializing vector by lenght
            self.lenght = vec_or_len
            self.vec = SparseBitString(vec_or_len)
            
        elif type(vec_or_len) is np.ndarray:   # Initializing vector by content
            self.lenght = len(vec_or_len)
            self.vec = vec_or_len
            
        else:
            raise TypeError("Input has to be integer (length) or numpy array")
        
        # If a string label is given, the initialized vector is added in the global catalog
        if label in Memory[1]:
            raise NameError("Label '" + str(label) + "'is already in catalog!")
        elif label:
            Memory[1].append(label)
            Memory[2].append(pointer)
            if len(Memory[0]) != 0:
                Memory[0] = np.concatenate((Memory[0], self.vec.reshape((1,N))))
            else:
                Memory[0] = self.vec.reshape((1,N))
            Dict[label] = self   # label is linked to vector -> (added to global dictionary)
            
        self.label = label       # assigning vector label
        self.pointer = pointer   # assigning pointer (vector)
        
        # Index of this object in global Memory...
        self.index = len(Memory[0]) - 1
    
    
    # Getters
    def getVec(self):   # it returns the array of the HD vector object
        return self.vec
    
    
    def getLabelID (self, num = N): # return label, it can be its assigned label or a list of the 'num' closest vectors labels
        if self.label:  # if it has an assigned label
            return [[self.label, 0]]
        
        else: #This vector hasn't a label. We find the 'num' most similar vectors in dictionary return a list of them
            HamVec = np.array([self.dist_vec(x) for x in Memory[0]]) #Distance to all vectors in memory
            # Indices where HamVec[i] < similarity threshold
            Indices = np.where(HamVec < thr)[0]
            # Cutting list to 'num'
            Indices = Indices[:num]
            # Result is a list of lists [label, distance]
            L = [[Memory[1][i], HamVec[i]] for i in Indices]
            return sorted(L, key = lambda r: r[1])
                
    def getLabelSP(self, num = N):   #get label of vector by searching in definitions...
        # Collect indeces where there's an actual pointer vector...
        Index = [i for i,x in enumerate(Memory[2]) if type(x) == np.ndarray]
        HamVec = np.array([self.dist_vec(Memory[2][x]) for x in Index])
        Indices = np.where(HamVec < thr)[0]  # Indices below similarity threshold
        Indices = Indices[:num]
        
        #Result is a list of lists [label,distance]
        L = [[Memory[1][Index[i]], HamVec[i]] for i in Indices]
        return sorted(L, key = lambda r: r[1])
        
    def getPointer(self):
        return self.pointer
    
    # Setters
    def setContent(self, in_array): #Set a new content
        self.vec = in_array
    def setPointer(self, other):    #Set a vector to point to
        self.pointer = other
        Memory[2][self.index] = other.getVec()  #setting new vector..
    
    # Distance
    def dist(self, other):        #Measure distance between two object vectors
        assert self.lenght == other.lenght    #Sanity check
        return np.count_nonzero(self.vec != other.vec)
    def dist_vec(self, vecc):      #Measure distance between an object vector and a numpy array (sometimes useful)
        assert self.lenght == len(vecc)      #Sanity check
        return np.count_nonzero(self.vec != vecc)
    
    # Arithmetic 
    def p(self, times):     #Pemutation aka rolling
        return HDvector(np.roll(self.vec, times))
    def ip(self, times):    #Inverse permutation aka inverse rolling
        return HDvector(np.roll(self.vec, self.lenght - times))
    def __add__(self, other): #Add
        return ADD(self, other)
    def __mul__(self, other):  #Multiplication
        return HDvector( np.bitwise_xor(self.vec, other.vec))
    def __pow__(self, other):  #Function that multiplies self's pointer by other's vector
        if isinstance(self.getPointer(), HDvector):
            return HDvector(np.bitwise_xor(self.getPointer().getVec(), other.vec))
    # Other functions
    def conc (self, other):  #Concatenate two vector objects arrays
        return HDvector(np.concatenate((self.vec, other.vec)))
    def __str__(self):
        return str(self.vec)